토픽별 리뷰 정보 TF-IDF

리뷰 파일 로드 불가로 실행 불가 - 경로만 수정

결과 파일은 따로 없습니다.

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 모듈
from sklearn.feature_extraction.text import TfidfVectorizer

# 설정
REVIEW_FILE_PATH = "files/final_files/review_all.parquet"
OUTPUT_DIR = "files/review_tfidf_analysis"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# 리뷰 데이터
df_review = pd.read_parquet(REVIEW_FILE_PATH)

# BERTopic 결과 파일들 (앞에서 생성한 파일들)

# 드라마 토픽 맵
df_hit_drama = pd.read_parquet('files/bertopic_results/drama_hit/drama_hit_topics.parquet')
df_flop_drama = pd.read_parquet('files/bertopic_results/drama_flop/drama_flop_topics.parquet')

# 영화 토픽 맵
df_hit_movie = pd.read_parquet('files/bertopic_results/movie_hit/movie_hit_topics.parquet')
df_flop_movie = pd.read_parquet('files/bertopic_results/movie_flop/movie_flop_topics.parquet')

# 클러스터 정보 로드 (있다면)
try:
    df_hit_drama_cluster = pd.read_csv('files/bertopic_results/drama_hit/topic_clusters.csv')
    df_flop_drama_cluster = pd.read_csv('files/bertopic_results/drama_flop/topic_clusters.csv')
    df_hit_movie_cluster = pd.read_csv('files/bertopic_results/movie_hit/topic_clusters.csv')
    df_flop_movie_cluster = pd.read_csv('files/bertopic_results/movie_flop/topic_clusters.csv')

    # topic → cluster 매핑 생성
    topic_to_cluster_hit_drama = dict(zip(df_hit_drama_cluster['topic_num'], df_hit_drama_cluster['cluster']))
    topic_to_cluster_flop_drama = dict(zip(df_flop_drama_cluster['topic_num'], df_flop_drama_cluster['cluster']))
    topic_to_cluster_hit_movie = dict(zip(df_hit_movie_cluster['topic_num'], df_hit_movie_cluster['cluster']))
    topic_to_cluster_flop_movie = dict(zip(df_flop_movie_cluster['topic_num'], df_flop_movie_cluster['cluster']))

    # 토픽 맵에 클러스터 정보 추가
    df_hit_drama['cluster'] = df_hit_drama['topic'].map(topic_to_cluster_hit_drama)
    df_flop_drama['cluster'] = df_flop_drama['topic'].map(topic_to_cluster_flop_drama)
    df_hit_movie['cluster'] = df_hit_movie['topic'].map(topic_to_cluster_hit_movie)
    df_flop_movie['cluster'] = df_flop_movie['topic'].map(topic_to_cluster_flop_movie)

except Exception as e:
    # 클러스터 정보가 없으면 topic을 cluster로 사용
    df_hit_drama['cluster'] = df_hit_drama['topic']
    df_flop_drama['cluster'] = df_flop_drama['topic']
    df_hit_movie['cluster'] = df_hit_movie['topic']
    df_flop_movie['cluster'] = df_flop_movie['topic']

In [ ]:
def get_top_tfidf_terms(tfidf_matrix, feature_names, top_n=20):
    """
    TF-IDF 매트릭스에서 상위 N개 단어 추출

    Args:
        tfidf_matrix: TF-IDF 행렬
        feature_names: 특성 이름 (단어)
        top_n: 추출할 상위 단어 개수

    Returns:
        list: (단어, 점수) 튜플 리스트
    """
    mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
    top_indices = mean_tfidf.argsort()[-top_n:][::-1]

    top_terms = [(feature_names[i], mean_tfidf[i]) for i in top_indices]
    return top_terms


def analyze_sentiment_tfidf(df_review, df_topic_map, content_type, hit_status, group_by='topic'):
    """
    특정 카테고리에 대해 토픽/클러스터별 sentiment 분석

    Args:
        df_review: 리뷰 데이터프레임
        df_topic_map: 토픽 매핑 데이터프레임
        content_type: 'movie' 또는 'drama'
        hit_status: 'hit' 또는 'flop'
        group_by: 'topic' 또는 'cluster'

    Returns:
        dict: 분석 결과
    """
    print(f"\n{'='*80}")
    print(f"🔍 {hit_status.upper()} {content_type.upper()} - Group by {group_by.upper()}")
    print(f"{'='*80}")

    # 리뷰와 토픽 맵 병합
    df_merged = df_review.merge(
        df_topic_map[['imdb_id', 'topic', 'cluster']],
        on='imdb_id',
        how='inner'
    )

    results = {}
    groups = df_merged[group_by].unique()
    groups = [g for g in groups if g != -1]  # outlier 제외

    for group_id in sorted(groups):
        group_data = df_merged[df_merged[group_by] == group_id].copy()

        # 최소 리뷰 개수 확인
        if len(group_data) < 40:
            continue

        print(f"\n--- {group_by.capitalize()} {group_id} ---")
        print(f"   총 리뷰: {len(group_data):,}개")

        # sentiment_score로 정렬
        group_data_sorted = group_data.sort_values('sentiment_score', ascending=False)
        top20 = group_data_sorted.head(20)
        bottom20 = group_data_sorted.tail(20)

        print(f"   긍정(Top 20) 범위: {top20['sentiment_score'].min():.4f} ~ {top20['sentiment_score'].max():.4f}")
        print(f"   부정(Bottom 20) 범위: {bottom20['sentiment_score'].min():.4f} ~ {bottom20['sentiment_score'].max():.4f}")

        results[group_id] = {}

        # Top20 TF-IDF 분석 (긍정 리뷰)
        try:
            vectorizer_top = TfidfVectorizer(
                stop_words='english',
                ngram_range=(2, 2),  # 2-gram
                min_df=2,
                max_df=0.9,
                token_pattern=r'\b[a-zA-Z]{3,}\b',
                max_features=1500
            )
            tfidf_matrix_top = vectorizer_top.fit_transform(top20['review_text_clean'])
            feature_names_top = vectorizer_top.get_feature_names_out()
            top_terms_top20 = get_top_tfidf_terms(tfidf_matrix_top, feature_names_top, top_n=20)

            for term, score in top_terms_top20[:5]:
                print(f"      • {term}: {score:.4f}")

            results[group_id]['top20'] = {
                'terms': top_terms_top20,
                'sentiment_range': (top20['sentiment_score'].min(), top20['sentiment_score'].max()),
                'avg_sentiment': top20['sentiment_score'].mean()
            }
        except Exception as e:
            results[group_id]['top20'] = None

        # Bottom20 TF-IDF 분석 (부정 리뷰)
        try:
            vectorizer_bottom = TfidfVectorizer(
                stop_words='english',
                ngram_range=(2, 2),
                min_df=2,
                max_df=0.9,
                token_pattern=r'\b[a-zA-Z]{3,}\b',
                max_features=1500
            )
            tfidf_matrix_bottom = vectorizer_bottom.fit_transform(bottom20['review_text_clean'])
            feature_names_bottom = vectorizer_bottom.get_feature_names_out()
            top_terms_bottom20 = get_top_tfidf_terms(tfidf_matrix_bottom, feature_names_bottom, top_n=20)

            print(f"\n 부정 리뷰 상위 TF-IDF 단어:")
            for term, score in top_terms_bottom20[:5]:
                print(f"      • {term}: {score:.4f}")

            results[group_id]['bottom20'] = {
                'terms': top_terms_bottom20,
                'sentiment_range': (bottom20['sentiment_score'].min(), bottom20['sentiment_score'].max()),
                'avg_sentiment': bottom20['sentiment_score'].mean()
            }
        except Exception as e:
            results[group_id]['bottom20'] = None

    return results

In [ ]:
print("\n" + "="*80)
print("🚀 전체 분석 시작")
print("="*80)

all_results = {}

# 1. Hit Drama - Topic
all_results['hit_drama_topic'] = analyze_sentiment_tfidf(
    df_review, df_hit_drama, 'drama', 'hit', group_by='topic'
)

# 2. Hit Drama - Cluster
all_results['hit_drama_cluster'] = analyze_sentiment_tfidf(
    df_review, df_hit_drama, 'drama', 'hit', group_by='cluster'
)

# 3. Flop Drama - Topic
all_results['flop_drama_topic'] = analyze_sentiment_tfidf(
    df_review, df_flop_drama, 'drama', 'flop', group_by='topic'
)

# 4. Flop Drama - Cluster
all_results['flop_drama_cluster'] = analyze_sentiment_tfidf(
    df_review, df_flop_drama, 'drama', 'flop', group_by='cluster'
)

# 5. Hit Movie - Topic
all_results['hit_movie_topic'] = analyze_sentiment_tfidf(
    df_review, df_hit_movie, 'movie', 'hit', group_by='topic'
)

# 6. Hit Movie - Cluster
all_results['hit_movie_cluster'] = analyze_sentiment_tfidf(
    df_review, df_hit_movie, 'movie', 'hit', group_by='cluster'
)

# 7. Flop Movie - Topic
all_results['flop_movie_topic'] = analyze_sentiment_tfidf(
    df_review, df_flop_movie, 'movie', 'flop', group_by='topic'
)

# 8. Flop Movie - Cluster
all_results['flop_movie_cluster'] = analyze_sentiment_tfidf(
    df_review, df_flop_movie, 'movie', 'flop', group_by='cluster'
)

In [ ]:
# 카테고리별 개별 파일 저장
for category, results in all_results.items():
    rows = []

    for group_id, group_results in results.items():
        if group_results is None:
            continue

        # Top20 결과 (긍정)
        if group_results.get('top20'):
            for rank, (term, score) in enumerate(group_results['top20']['terms'], 1):
                rows.append({
                    'category': category,
                    'group_type': category.split('_')[-1],  # 'topic' or 'cluster'
                    'group_id': group_id,
                    'sentiment_type': 'high',
                    'rank': rank,
                    'term': term,
                    'tfidf_score': score,
                    'avg_sentiment': group_results['top20']['avg_sentiment'],
                    'min_sentiment': group_results['top20']['sentiment_range'][0],
                    'max_sentiment': group_results['top20']['sentiment_range'][1]
                })

        # Bottom20 결과 (부정)
        if group_results.get('bottom20'):
            for rank, (term, score) in enumerate(group_results['bottom20']['terms'], 1):
                rows.append({
                    'category': category,
                    'group_type': category.split('_')[-1],
                    'group_id': group_id,
                    'sentiment_type': 'low',
                    'rank': rank,
                    'term': term,
                    'tfidf_score': score,
                    'avg_sentiment': group_results['bottom20']['avg_sentiment'],
                    'min_sentiment': group_results['bottom20']['sentiment_range'][0],
                    'max_sentiment': group_results['bottom20']['sentiment_range'][1]
                })

    if rows:
        df_result = pd.DataFrame(rows)
        output_file = f'{OUTPUT_DIR}/{category}_tfidf_results.csv'
        df_result.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"{output_file} ({len(df_result):,}행)")

In [ ]:
# 전체 결과를 하나의 파일로 저장
all_rows = []

for category, results in all_results.items():
    for group_id, group_results in results.items():
        if group_results is None:
            continue

        # Top20
        if group_results.get('top20'):
            for rank, (term, score) in enumerate(group_results['top20']['terms'], 1):
                all_rows.append({
                    'category': category,
                    'group_type': category.split('_')[-1],
                    'group_id': group_id,
                    'sentiment_type': 'high',
                    'rank': rank,
                    'term': term,
                    'tfidf_score': score,
                    'avg_sentiment': group_results['top20']['avg_sentiment'],
                    'min_sentiment': group_results['top20']['sentiment_range'][0],
                    'max_sentiment': group_results['top20']['sentiment_range'][1]
                })

        # Bottom20
        if group_results.get('bottom20'):
            for rank, (term, score) in enumerate(group_results['bottom20']['terms'], 1):
                all_rows.append({
                    'category': category,
                    'group_type': category.split('_')[-1],
                    'group_id': group_id,
                    'sentiment_type': 'low',
                    'rank': rank,
                    'term': term,
                    'tfidf_score': score,
                    'avg_sentiment': group_results['bottom20']['avg_sentiment'],
                    'min_sentiment': group_results['bottom20']['sentiment_range'][0],
                    'max_sentiment': group_results['bottom20']['sentiment_range'][1]
                })

if all_rows:
    df_all_results = pd.DataFrame(all_rows)
    output_file = f'{OUTPUT_DIR}/all_sentiment_tfidf_results.csv'
    df_all_results.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"통합 결과 저장: {output_file} ({len(df_all_results):,}행)")
else:
    print("저장할 결과 없음")
    df_all_results = pd.DataFrame()